# V3：两层

In [1]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
from queue import Queue
filepath_out='../output/bench3.router'
netlist_file_path = '../benchmark/bench3.nl'
gridfile_path  = '../benchmark/bench3.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_grid_original,layer2_grid_original = parse_gridfile(gridfile_path)

bend_penalty: 10 
 via_penalty: 100
size of layer1: (60, 60)
size of layer2: (60, 60)


In [2]:
# plot problem 根据grid文件可视化障碍物和每个格子的cost，同时标出source和target 目前支持到同一层S-T对的情况，显示两层
plot_problem("../output/bench3_problem.jpg",columns,rows,layer1_grid_original,layer2_grid_original,nets)

In [3]:
def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def mark_path_on_grid(layer1_grid, path):
    #print(path)
    for cell in path:
        x, y, _ = cell
        layer1_grid[x][y] = -1

In [4]:
def get_cell_cost(layer1_grid, cell,path_tmp,bend_penalty):
    x, y, layer = cell
    cell_cost = 1  # 默认的单元代价
    if layer1_grid[x][y] == -1:
        cell_cost = float('inf')  # -1表示无法通过的细胞
    elif layer1_grid[x][y] != 1:
        cell_cost = layer1_grid[x][y]  # 非单元代价
    if len(path_tmp) >= 2:
        prev_cell = path_tmp[-2]
        prev_x, prev_y, prev_layer = prev_cell
        if layer != prev_layer and (prev_x != x or prev_y != y):  # 方向变化
            cell_cost += bend_penalty
    return cell_cost

def expand_source_to_target(rows, columns, layer1_grid, source, target,bend_penalty):
    queue = Queue()
    visited = set()
    parents = {}
    costs = {}  # Store the cumulative costs for each cell
    
    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    queue.put(source_tuple)
    visited.add(source_tuple)
    costs[source_tuple] = 0  # Initial cost for the source cell is 0

    while not queue.empty():
        current_cell = queue.get()

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path,costs

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])
            path_tmp= reconstruct_path(source_tuple, current_cell, parents)
            # Calculate the cost to reach the neighbor cell
            cost = costs[current_cell] + get_cell_cost(layer1_grid, neighbor_tuple,path_tmp,bend_penalty)

            if neighbor_tuple not in visited or cost < costs[neighbor_tuple]:
                queue.put(neighbor_tuple)
                visited.add(neighbor_tuple)
                parents[neighbor_tuple] = current_cell
                costs[neighbor_tuple] = cost
    return []

# modified by junjun
def expand_source_to_target_jun(rows, columns, layer1_grid, source, target,bend_penalty):
    wavefront = {}
    visited = set()
    parents = {}
    costs = {}  # Store the cumulative costs for each cell
    
    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    wavefront[source_tuple] = 0
    costs[source_tuple] = 0  # Initial cost for the source cell is 0

    while wavefront:
        # get lowest cost cell on a wavefront structure
        current_cell = sorted(wavefront.items(),key=lambda s:int(s[1]))[0][0]

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path,costs

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])

            if neighbor_tuple not in visited:
                path_tmp= reconstruct_path(source_tuple, current_cell, parents)
                # Calculate the cost to reach the neighbor cell
                cost = costs[current_cell] + get_cell_cost(layer1_grid, neighbor_tuple,path_tmp,bend_penalty)
                
                # ignore blocks
                if cost!= np.inf:
                    if neighbor_tuple not in wavefront.keys() or costs[neighbor_tuple] > cost:
                        costs[neighbor_tuple] = cost
                        parents[neighbor_tuple] = current_cell

                    if neighbor_tuple not in wavefront.keys():
                        # add cell N to waveform, indexed by pathcost
                        wavefront[neighbor_tuple]=cost         

        visited.add(current_cell)    
        del wavefront[current_cell]                  
    return None,None

def get_neighbors(rows, columns, cell):
    x, y, layer = cell
    neighbors = []
    if x > 0:
        neighbors.append({'x': x - 1, 'y': y, 'layer': layer})
    if x < columns - 1:
        neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
    if y > 0:
        neighbors.append({'x': x, 'y': y - 1, 'layer': layer})
    if y < rows - 1:
        neighbors.append({'x': x, 'y': y + 1, 'layer': layer})

    return neighbors

In [5]:
def nounit_bend_penalty_cost_router(rows, columns, layer1_grid, nets,bend_penalty):
    routing_table = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer1_grid[pin1['x']][pin1['y']] = -1
        layer1_grid[pin2['x']][pin2['y']] = -1
        ## 防止布线在后续的pin上，先将所有的pin标记为-1；
        
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer1_grid[pin1['x']][pin1['y']] = 1
        layer1_grid[pin2['x']][pin2['y']] = 1
        path,costs = expand_source_to_target_jun(rows, columns, layer1_grid, pin1, pin2,bend_penalty) # 少了第二个return jun
        mark_path_on_grid(layer1_grid,path)
        routing_table[net_id] = path
    return routing_table,costs

In [6]:
layer1_grid = layer1_grid_original
routing_table,cost1 = nounit_bend_penalty_cost_router(rows,columns,layer1_grid.T,[x for x in nets if x['pin1']['layer']==1 and x['pin2']['layer']==1],bend_penalty)
print(routing_table)
#generate_output_file(filepath_out,net_num,routing_table)
plot_path('../output/bench3_v3_layer1.jpg',columns=columns,rows=rows,block_list=np.argwhere(layer1_grid.T==-1),path_dict=routing_table)

{1: [(2, 57, 1), (2, 56, 1), (2, 55, 1), (2, 54, 1), (2, 53, 1), (2, 52, 1), (2, 51, 1), (2, 50, 1), (2, 49, 1)], 3: [(3, 57, 1), (3, 56, 1), (3, 55, 1), (3, 54, 1), (3, 53, 1), (3, 52, 1), (3, 51, 1), (3, 50, 1), (3, 49, 1)], 5: [(4, 57, 1), (5, 57, 1), (5, 56, 1), (5, 55, 1), (5, 54, 1), (5, 53, 1), (5, 52, 1), (5, 51, 1), (5, 50, 1), (5, 49, 1)], 7: [(6, 57, 1), (7, 57, 1), (8, 57, 1), (8, 56, 1), (8, 55, 1), (8, 54, 1), (8, 53, 1), (8, 52, 1), (8, 51, 1), (8, 50, 1), (8, 49, 1)], 9: [(9, 57, 1), (10, 57, 1), (11, 57, 1), (12, 57, 1), (12, 56, 1), (12, 55, 1), (12, 54, 1), (12, 53, 1), (12, 52, 1), (12, 51, 1), (12, 50, 1), (12, 49, 1)], 11: [(1, 29, 1), (2, 29, 1), (3, 29, 1), (4, 29, 1), (5, 29, 1), (6, 29, 1), (7, 29, 1), (8, 29, 1), (9, 29, 1), (10, 29, 1), (11, 29, 1), (12, 29, 1), (13, 29, 1), (14, 29, 1), (15, 29, 1), (16, 29, 1), (17, 29, 1), (18, 29, 1), (19, 29, 1)], 13: [(21, 29, 1), (22, 29, 1), (23, 29, 1), (23, 28, 1), (23, 27, 1), (23, 26, 1), (23, 25, 1), (23, 24, 1)

In [7]:
if [x for x in nets if x['pin1']['layer']==2 and x['pin2']['layer']==2]:
    layer2_grid = layer2_grid_original
    routing_table2,cost2 = nounit_bend_penalty_cost_router(rows,columns,layer2_grid.T,[x for x in nets if x['pin1']['layer']==2 and x['pin2']['layer']==2],bend_penalty)
    print(routing_table2)
    #generate_output_file(filepath_out,net_num,routing_table2)
    plot_path('../output/bench3_v3_layer2.jpg',columns=columns,rows=rows,block_list=np.argwhere(layer2_grid.T==-1),path_dict=routing_table2)

{2: [(56, 56, 2), (55, 56, 2), (54, 56, 2), (53, 56, 2), (52, 56, 2), (51, 56, 2), (50, 56, 2), (49, 56, 2), (48, 56, 2), (47, 56, 2), (46, 56, 2), (45, 56, 2)], 4: [(56, 55, 2), (55, 55, 2), (54, 55, 2), (53, 55, 2), (52, 55, 2), (51, 55, 2), (50, 55, 2), (49, 55, 2), (48, 55, 2), (47, 55, 2), (46, 55, 2), (45, 55, 2)], 6: [(56, 54, 2), (55, 54, 2), (54, 54, 2), (53, 54, 2), (52, 54, 2), (52, 53, 2), (51, 53, 2), (50, 53, 2), (49, 53, 2), (48, 53, 2), (47, 53, 2), (46, 53, 2), (45, 53, 2)], 8: [(56, 52, 2), (55, 52, 2), (54, 52, 2), (53, 52, 2), (52, 52, 2), (52, 51, 2), (52, 50, 2), (51, 50, 2), (50, 50, 2), (49, 50, 2), (48, 50, 2), (47, 50, 2), (46, 50, 2), (45, 50, 2)], 10: [(56, 49, 2), (55, 49, 2), (54, 49, 2), (53, 49, 2), (52, 49, 2), (52, 48, 2), (52, 47, 2), (52, 46, 2), (51, 46, 2), (50, 46, 2), (49, 46, 2), (48, 46, 2), (47, 46, 2), (46, 46, 2), (45, 46, 2)], 12: [(9, 19, 2), (9, 18, 2), (9, 17, 2), (9, 16, 2), (9, 15, 2), (9, 14, 2), (9, 13, 2), (9, 12, 2), (9, 11, 2), (9

In [8]:
# 合并一下再输出 顺便按字典键从小到大排序
routing_table.update(routing_table2)
generate_output_file(filepath_out,net_num,dict(sorted(routing_table.items(),key=lambda x:x[0])))